In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 120

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn as skl

# make parent kaggle folder importable
import sys
import os
sys.path.append(os.path.abspath("../"))
import importlib
import graphs as g
import utils as u
import processing as p
import model as m
import selection as s

Using TensorFlow backend.


# Data

In [2]:
train_data = pd.read_csv('input/train.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404290 non-null object
question2       404288 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [3]:
train_data = train_data.dropna()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404288 entries, 0 to 404289
Data columns (total 6 columns):
id              404288 non-null int64
qid1            404288 non-null int64
qid2            404288 non-null int64
question1       404288 non-null object
question2       404288 non-null object
is_duplicate    404288 non-null int64
dtypes: int64(4), object(2)
memory usage: 21.6+ MB


In [4]:
train_data[train_data.qid1==8461]

,id,qid1,qid2,question1,question2,is_duplicate
4279,4279,8461,8462,How do I improve my English speaking?,How do I improve my pronunciation in English?,1
11300,11300,8461,10859,How do I improve my English speaking?,How can I specifically improve my English?,1
14828,14828,8461,11396,How do I improve my English speaking?,How can I improve my English speaking ability?,1
23369,23369,8461,2438,How do I improve my English speaking?,How can I improve my pronunciation in English?,1
27111,27111,8461,12214,How do I improve my English speaking?,How can I improve my pronunciation of English ...,1
31918,31918,8461,21693,How do I improve my English speaking?,How could I improve my English?,1
32544,32544,8461,55435,How do I improve my English speaking?,How can I improve my English in a very short t...,1
60570,60570,8461,3939,How do I improve my English speaking?,What is the best way to improve my spoken Engl...,1
90399,90399,8461,13756,How do I improve my English speaking?,How can I improve English speaking skill?,1
96548,96548,8461,11477,How do I improve my English speaking?,I want to improve my English?,1


In [17]:
test_data = pd.read_csv('input/test.csv')
test_data.info()
test_data = test_data.dropna()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345796 entries, 0 to 2345795
Data columns (total 3 columns):
test_id      int64
question1    object
question2    object
dtypes: int64(1), object(2)
memory usage: 53.7+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2345790 entries, 0 to 2345795
Data columns (total 3 columns):
test_id      int64
question1    object
question2    object
dtypes: int64(1), object(2)
memory usage: 71.6+ MB


In [18]:
importlib.reload(p)

data = train_data.question1.append(train_data.question2)
text = data.append(test_data.question1).append(test_data.question2).values
text_we = p.word_encoder(text, title='corpus', stop_words=[], max_features=2500)

text: 2500 


# Model

In [26]:
text_features = text_we.get_feature_names()

X1 = text_we.transform(train_data.question1.values)
X2 = text_we.transform(train_data.question2.values)
X = X1.merge(X2)

u.log('X1', X1.shape)
u.log('X2', X2.shape)
u.log('X', X.shape)

AttributeError: merge not found

In [9]:
y = train_data.is_duplicate
print(y.value_counts())
Y = p.onehot(y)

0    255025
1    149263
Name: is_duplicate, dtype: int64
Y: (404288, 2) 


In [14]:
model = m.keras_classification(X.shape[1], Y.shape[1])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.3, epochs=5, batch_size=10000)
model.save('quora-pairs.h5')

Train on 283001 samples, validate on 121287 samples
Epoch 1/10
283001/283001 [==============================] - 122s - loss: 0.6077 - acc: 0.6753 - val_loss: 0.5649 - val_acc: 0.7140

# Submission

In [4]:
import time

sample = pd.read_csv('input/sample_submission.csv')
print(sample.shape)

submission = pd.DataFrame(columns=['test_id', 'is_duplicate'])
submission.test_id = sample.test_id
submission.is_duplicate = np.random.rand(sample.shape[0])
submission.info()

filename = 'submission-' + time.strftime("%Y%m%d%H%M", time.gmtime()) + '.csv'
submission.to_csv(filename, index=False)

(2345796, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345796 entries, 0 to 2345795
Data columns (total 2 columns):
test_id         int64
is_duplicate    float64
dtypes: float64(1), int64(1)
memory usage: 35.8 MB
